In [11]:
import numpy as np
import quaternion
import spatialmath as sm
import yaml

import rospy
from geometry_msgs.msg import Vector3Stamped
from hr_recycler_msgs.msg import ToolType
import hrr_cobot_robot
import hrr_common

import sim_robots

## Evaluate Poses against reachability

Use `sim_robots` package when ROS is not available.

### Preliminaries

* the `md` handle is packed in hrr_cobot_robot.HrrCobotControl, usually `cobot`, via the `get_model()` function
* the `forward_kin()` functoin is identical to `cobot.FK()`
* the `IK()` function is identical to `cobot.IK()`
* the calculation uses `numba` and takes quite some time to load and compile all functions on first call, i.e. every time the `md` handle is created / instantiated.
* the `IK()` resolves **EE-poses** not **TCP-poses**
* the `legal_joint_config` function is copied from `hrr_cobot_contorl.py`

Thus the `DummyCobot` can be used as a replacement of the actual cobot handle.

In [2]:
cobot_version = ToolType.SHAFT_GRINDER

In [3]:
if cobot_version > 0:
    urdf = sim_robots.get_racer_path() / f'hrr_cobot.t{cobot_version}.urdf'
    xml = sim_robots.get_racer_path() / f'hrr_cobot.t{cobot_version}.xml'
else:
    urdf = sim_robots.get_racer_path() / f'hrr_cobot.urdf'
    xml = sim_robotsget_racer_path() / f'hrr_simple.xml'

In [ ]:
md = sim_robots.SymbolicRacerCobot.from_urdf(urdf)

In [ ]:
md.IK(md.forward_kin(md.sample_q()))

In [ ]:
class DummyCobot:
    
    def __init__(self, md):
        self._md = md
        self.q = np.zeros(6)
        # adjust to current setting
        self._joint_limits_soft_dist = np.r_[0.05 * np.ones(4), 0.22, 0.1]
        self._joint_limits_hard_dist = np.r_[0.01 * np.ones(4), 0.1, 0.01]
    
    def IK(self, *a, **kw):
        return self._md.IK(*a, **kw)
    
    def FK(self, *a, **kw):
        return self._md.forward_kin(*a, **kw)
    
    @property
    def joint_limits(self):
        return self._md.joint_limits
    
    def is_reachable(self, T_B_E_des, check_closest_only=False, validate_closest=False, log=True):
        try:
            q_ik = self.IK(T_B_E_des)
            q_ik = q_ik[np.argsort(np.linalg.norm(q_ik - self.q, axis=1))]
            lb_check = q_ik >= (cobot.joint_limits[:,0] + cobot._joint_limits_hard_dist)
            ub_check = q_ik <= (cobot.joint_limits[:,1] - cobot._joint_limits_hard_dist)
            valid = np.logical_and(lb_check, ub_check)
            if np.all(valid):
                return True
            res = np.any(np.all(valid, axis=1))
            if not res:
                return res
            if check_closest_only:
                return np.all(valid[0,:])
            jnts = [q_i + 1 for q_i in np.where(np.any(~valid, axis=0))[0]]
            rospy.logwarn(f"IK contains invalid configurations for joint(s): {jnts}")
            return res
        except (IndexError, ValueError):
            rospy.logerr(f'no solution found for pose:\n{T_B_E_des}')
            return False
    
cobot = DummyCobot(md)

## usage examples

In [ ]:
T_legal = cobot.FK(md.sample_q())
cobot.is_reachable(T_legal)

In [ ]:
T_illegal = sm.SE3(0, 0, 1.5) @ T_legal
cobot.is_reachable(T_illegal)

## Evaluations

### Check joint setting from panel

In [ ]:
qs = [
    np.deg2rad(np.array([-75.174, -10.239, -144.15, 165.88, 50.026, -86.01])),
    np.deg2rad(np.array([-17.647, -10.247, -150., 165.875, 50.025, -86.007])),
    np.deg2rad(np.array([-42.888, 3.350, -94.271, 97.524, 99.807, 0.539])), 
    np.deg2rad(np.array([-42.888, 3.350, -94.271, 97.524, 125.807, 0.539])),
    np.deg2rad(np.array([-59.125, 34.703, -103.347, -54.237, -63.839, 118.01])),
    np.r_[0.555,  1.14,  -0.946, -1.343,  2.033, -3.367]
]

In [ ]:
dq = cobot2.joint_limit_distance()
cobot2.joint_limit_avoidance_needed(d_limit=cobot._joint_limits_hard_dist, dq=dq) 

In [ ]:
cobot.q = np.zeros(6)
[cobot2.is_reachable(cobot.FK(q), log=False) for q in qs]

## Check with real robot

In [4]:
hrr_common.set_ros_environment("hrrcobotLinux54")

current hostname:	pizza
current IP:      	129.187.147.180
ROS-MASTER-URI:  	http://hrrcobotLinux54:11311


In [5]:
rospy.init_node("test_real")

In [6]:
cobot2 = hrr_cobot_robot.HrrCobotControl.from_ros(cobot_prefix="/hrr_cobot")

[ERROR] [1647466437.076599]: source directory of motion planner /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src does not exist


[INFO] [1647466444.709016]: some functions are not yet compiled. Expect delays upon first call


In [ ]:
## Check poses

poses = []

def vecst_dict_to_np(yaml_dict):
    v = yaml_dict['vector']
    return np.r_[v['x'], v['y'], v['z']]
    
msg_1 = yaml.safe_load('''
start_location:
  header:
    seq: 0
    stamp:
      secs: 1647355702
      nsecs: 952467918
    frame_id: "base_link"
  vector:
    x: 0.403009518832
    y: 0.0310273300235
    z: 0.0181086506694
end_location:
  header:
    seq: 0
    stamp:
      secs: 1647355702
      nsecs: 953128099
    frame_id: "base_link"
  vector:
    x: 0.404239246231
    y: 0.0627181001677
    z: 0.0215861446855
''')

msg_2 = yaml.safe_load('''
start_location:
  header:
    seq: 0
    stamp:
      secs: 1647367359
      nsecs: 837783098
    frame_id: "base_link"
  vector:
    x: 0.516543880922
    y: 0.151079536263
    z: 0.03
end_location:
  header:
    seq: 0
    stamp:
      secs: 1647367359
      nsecs: 838419914
    frame_id: "base_link"
  vector:
    x: 0.516755472499
    y: 0.192494410614
    z: 0.03
''')                       

In [ ]:
def test_msg_and_key(msg, key):
    T_B_C_des = hrr_common.calc_goal_pose(normal=np.r_[0,0,1], y_axis=np.r_[0, 1, 0], 
                                          p_location=vecst_dict_to_np(msg[key]))
    return cobot2.is_reachable(T_B_C_des @ cobot2.T_E_C_robot.inv())

In [ ]:
test_msg_and_key(msg_1, 'end_location'), test_msg_and_key(msg_1, 'start_location'), \
test_msg_and_key(msg_2, 'end_location'), test_msg_and_key(msg_2, 'start_location')

## hackish solution to get target pose with rotary symmetry

In [7]:
B_normal = np.r_[0.0, -0.08715574274765817, 0.9961946980917455]
B_normal /= np.linalg.norm(B_normal)

In [8]:
T_test = cobot2.get_valid_ee_pose(np.r_[0.7, -0.25, 0.0531], B_normal=B_normal)
if T_test is None:
    rospy.logerr("fuck off")
T_test

IK q1 not in cache. Regenerate  


  -0.6686    0.6686   -0.3255    0.7506    
   0.2909   -0.1676   -0.9419   -0.1118    
  -0.6844   -0.7245   -0.08241   0.1601    
   0         0         0         1         
